# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf

# Import API key
from api_keys import geoapify_key

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,82.38,89,75,9.22,CC,1682816086
1,1,yoloten,37.2989,62.3597,73.31,13,99,9.40,TM,1682816086
2,2,colonia,50.9333,6.9500,48.96,93,100,6.91,DE,1682815882
3,3,waitangi,-43.9535,-176.5597,59.83,87,99,4.00,NZ,1682816086
4,4,wainwright,52.8390,-110.8572,55.15,34,38,8.97,CA,1682816086


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [59]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    alpha = 0.5,
    tiles = "CartoDark",
    frame_height = 500,
    frame_width = 700,
    xlabel = "Longitude",
    ylabel = "Latitude",
)

# Other tile options
# EsriNatGeo
# EsriImagery
# CartoDark
# OSM

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
# Units are imperial so they will vary from the assignment instructions
nice_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & 
                                   (city_data_df["Max Temp"] < 85) & 
                                   (city_data_df["Cloudiness"] < 50 ) & 
                                   (city_data_df["Wind Speed"] < 12)]

nice_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,tamanrasset,22.7850,5.5228,75.11,17,0,5.75,DZ,1682815917
28,28,carutapera,-1.1950,-46.0200,76.59,92,3,5.53,BR,1682816090
35,35,dwarka,22.2394,68.9678,79.34,67,0,7.52,IN,1682816091
54,54,daru,-9.0763,143.2092,82.02,82,14,7.43,PG,1682816095
89,89,howrah,22.5892,88.3103,78.80,83,40,6.91,IN,1682816100


In [61]:
# Drop any rows with null values
nice_weather_df.dropna()

# Display sample data
nice_weather_map = nice_weather_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    alpha = 0.5,
    tiles = "CartoDark",
    frame_height = 500,
    frame_width = 700,
    xlabel = "Longitude",
    ylabel = "Latitude",
)
nice_weather_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,23,tamanrasset,22.7850,5.5228,75.11,17,0,5.75,DZ,1682815917,
28,28,carutapera,-1.1950,-46.0200,76.59,92,3,5.53,BR,1682816090,
35,35,dwarka,22.2394,68.9678,79.34,67,0,7.52,IN,1682816091,
54,54,daru,-9.0763,143.2092,82.02,82,14,7.43,PG,1682816095,
89,89,howrah,22.5892,88.3103,78.80,83,40,6.91,IN,1682816100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
params = { 
         "radius": radius,
          "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"] 
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and 
    # longitude to the params dictionary
    params["categories"] = f"accommodation.hotel"
    params["filter"] =  f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df.head()

Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
carutapera - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwarika Hotel
daru - nearest hotel: Kuki
howrah - nearest hotel: Sun India Guest House
georgetown - nearest hotel: Page 63 hostel
doka - nearest hotel: No hotel found
abilene - nearest hotel: Abilene Hotel
san julian - nearest hotel: No hotel found
ar ruways - nearest hotel: No hotel found
padang - nearest hotel: losmen Surya
crane - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
santa cruz - nearest hotel: Aviador Hotel Boutique
touros - nearest hotel: Pousada Atlântico
la mira - nearest hotel: Hotel La Loma
tura - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
jiwani - nearest hotel: No hotel found
sarupathar - nearest hotel: No hotel found
noumea - nearest hotel: Hotel Gondwana
uthal - nearest hotel: No hotel found
bel ombre - nearest hotel: Villa La Cachette
trinidad - nearest hotel: Hotel Palace
bandar abbas - ne

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,23,tamanrasset,22.7850,5.5228,75.11,17,0,5.75,DZ,1682815917,فندق الأمان
28,28,carutapera,-1.1950,-46.0200,76.59,92,3,5.53,BR,1682816090,No hotel found
35,35,dwarka,22.2394,68.9678,79.34,67,0,7.52,IN,1682816091,The Dwarika Hotel
54,54,daru,-9.0763,143.2092,82.02,82,14,7.43,PG,1682816095,Kuki
89,89,howrah,22.5892,88.3103,78.80,83,40,6.91,IN,1682816100,Sun India Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
# write the DataFrame to a CSV file
hotel_df.to_csv("output_data/hotels.csv")

# Display sample data
# Use hover_cols to add additional information to the hover pop-up
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    alpha = 0.75,
    size = "Humidity",
    tiles = "CartoDark",
    frame_height = 500,
    frame_width = 700,
    xlabel = "Longitude",
    ylabel = "Latitude",
    hover_cols = ["Humidity", "City", "Country", "Hotel Name"],
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)